<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/mobile2.png?raw=true" width=800 align="left"/>

# 실습 내용

- 여러 알고리즘으로 만든 모델의 성능을 교차 검증을 통해 예측해 봅니다.
- 성능이 좋을 것으로 예상되는 알고리즘으로 최적화된 성능의 모델을 만듭니다.
- 튜닝된 모델의 성능을 평가합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data['CHURN'] = data['CHURN'].map({'STAY':0, 'LEAVE': 1})

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**데이터 설명**

- COLLEGE: 대학 졸업여부
- INCOME: 연수입
- OVERAGE: 월평균 초과사용 시간(분)
- LEFTOVER: 월평균 잔여시간비율(%)
- HOUSE: 집값
- HANDSET_PRICE: 스마트폰 가격
- OVER_15MINS_CALLS_PER_MONTH: 월평균 장기통화(15분이상) 횟수
- AVERAGE_CALL_DURATION: 평균 통화 시간
- REPORTED_SATISFACTION: 만족도 설문조사 결과
- REPORTED_USAGE_LEVEL: 사용도 자가진단 결과
- CONSIDERING_CHANGE_OF_PLAN: 향후 변경계획 설문조사 결과
- **CHURN: 이탈(번호이동) 여부**

In [ ]:
# 데이터 살펴보기
data.head()

In [ ]:
# 기술통계 확인
data.describe()

In [ ]:
# 변수 관련 정보 확인
data.info()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

In [ ]:
# 변수 제거
drop_cols = ['id']
data.drop(columns=drop_cols, inplace=True)

# 확인
data.head()

**2) x, y 분리**

In [ ]:
# x, y 분리
target = 'CHURN'
x = data.drop(columns=target)
y = data[target]

**3) 가변수화**

In [ ]:
# 가변수화
dumm_cols =['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**4) 학습용, 평가용 데이터 분리**

In [ ]:
# 학습용, 평가용 데이터 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

**5) 정규화**

In [ ]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_test_s = scaler.transform(x_test)

# 4.성능 예측

- 여러 알고리즘으로 만든 모델의 성능을 교차 검증을 통해 예측해 봅니다.
- 하이퍼파라미터를 설정하지 않고 진행합니다.
- 예측된 성능 정보를 수집해 마지막에 비교합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

**1) KNN**

- KNN 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [2]:
# 성능예측
cv_score = cross_val_score(KNeighborsClassifier(), x_train_s, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

NameError: name 'cross_val_score' is not defined

In [ ]:
# 결과수집
result = {}
result['KNN'] = cv_score.mean()

**2) Decision Tree**

- Decision Tree 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [ ]:
# 성능예측
cv_score = cross_val_score(DecisionTreeClassifier(), x_train, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Decision Tree'] = cv_score.mean()

**3) Logistic Regression**

- Logistic Regression 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [ ]:
# 성능예측
cv_score = cross_val_score(LogisticRegression(), x_train, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Logistic Regression'] = cv_score.mean()

**4) Random Forest**

- Random Forest 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [ ]:
# 성능예측
cv_score = cross_val_score(RandomForestClassifier(), x_train, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['Random Forest'] = cv_score.mean()

**5) XGBoost**

- XGBoost 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [ ]:
# 성능예측
cv_score = cross_val_score(XGBClassifier(), x_train, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['XGBoost'] = cv_score.mean()

**6) LightGBM**

- LightGBM 모델의 성능을 교차 검증을 통해 예측해 봅니다.

In [ ]:
# 성능예측
cv_score = cross_val_score(LGBMClassifier(), x_train, y_train, cv=5)

# 결과확인
print('평균:', cv_score.mean())

In [ ]:
# 결과수집
result['LightGBM'] = cv_score.mean()

# 5.결과 확인

- 예측된 각 모델의 성능을 비교합니다.

In [ ]:
# 성능 비교
print('=' * 40)
for m_name, score in result.items():
    print(m_name, score.round(3))
print('=' * 40)

In [ ]:
# 성능 시각화 비교
plt.figure(figsize=(5, 5))
plt.barh(list(result), result.values())
plt.show()

# 6.성능 튜닝

- 위에서 성능이 가장 좋을 것으로 예측된 알고리즘으로 모델링합니다.
- 본 실습에서는 하이퍼파라미터가 최적화된 LightGBM 모델을 만듭니다.

In [ ]:
# 파라미터 
param = {'max_depth': range(1, 21),
         'n_estimators': range(60, 131, 10)}

# 모델 선언
model = GridSearchCV(LGBMClassifier(verbose=-1),
                     param,
                     cv=5,
                     scoring='accuracy')

In [ ]:
# 학습하기(많은 시간이 소요될 수 있음)
model.fit(x_train, y_train)

In [ ]:
# 최적 파라미터, 예측 최고 성능
print(model.best_params_)
print(model.best_score_)

In [ ]:
# 변수 중요도 시각화
plt.figure(figsize=(6, 5))
plt.barh(list(x), model.best_estimator_.feature_importances_)
plt.show()

# 7.성능 평가

- 최적 파라미터로 학습된 모델에 대해 최종 성능 평가를 진행합니다.

In [ ]:
# 예측하기
y_pred = model.predict(x_test)

In [ ]:
# 성능평가
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))